<a href="https://colab.research.google.com/github/CosmicSelection/DuelSim/blob/main/DuelSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install yugioh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import random as rand
import re
import yugioh
import copy

class onField:
  def __init__(self, card, pos, isFaceUp):
    self.pos = pos
    self.isFaceUp = isFaceUp
    self.name = card.name
    self.type = card.type
    self.description = card.description
    cardType = re.search("monster", card.type.lower())
    if cardType != None:
      self.attack = card.attack
      self.bonus_atk = 0
      self.total_atk = self.attack + self.bonus_atk
      self.defense = card.defense
      self.bonus_def = 0
      self.total_def = self.defense + self.bonus_def
      self.race = card.race
      self.level = card.level
      self.attribute = card.attribute
      self.posChanged = True
      self.equipped = []
      self.can_atk = True
      self.switched = False
  def __str__(self):
    cardType = re.search("Monster", self.type)
    if cardType == None:
      return f"{self.name} ({self.type}: {self.description})"
    elif self.pos == "attack":
      return f"{self.name} ({self.total_atk} atk, {self.total_def} def)"
    else:
      return f"{self.name} ({self.total_def} def, {self.total_atk} atk)"
  def increaseBonusAtk(self,bonus_attVal):
    self.bonus_atk += bonus_attVal
    self.total_atk = self.attack + self.bonus_atk
  def increaseBonusDef(self,bonus_defVal):
    self.bonus_def += bonus_defVal
    self.total_def = self.defense + self.bonus_def    
def atk_pts(monster):
  return monster.total_atk
def def_pts(monster):
  return monster.total_def



In [17]:
p1DeckBuild = np.array(["Mystical Elf", "Winged Dragon, Guardian of the Fortress #1", "Summoned Skull", "Beaver Warrior", "Dark Magician",
                        "Gaia The Fierce Knight", "Curse of Dragon", "Celtic Guardian", "Mammoth Graveyard", "Silver Fang",
                        "Giant Soldier of Stone", "Dragon Zombie", "Doma The Angel of Silence", "Ansatsu", "Witty Phantom", "Claw Reacher", 	
                        "Sword of Dark Destruction", "Book of Secret Arts", "Dark Hole", "Dian Keto the Cure Master", "Ancient Elf",
                        "Magical Ghost", "Fissure", "Trap Hole", "De-Spell", "Monster Reborn", "Reinforcements", "Change of Heart",
                        "Wall of Illusion", "Neo the Magic Swordsman", "Baron of the Fiend Sword", "Sorcerer of the Doomed", 	"Waboku", "Dian Keto the Cure Master", 
                        "Magic Cylinder", "Sakuretsu Armor", "Remove Trap", "Castle Walls", "Ultimate Offering", "Raigeki"])
                        
p2DeckBuild = np.array(["Blue-Eyes White Dragon", "Hitotsu-Me Giant", 	"Battle Ox", 	"Koumori Dragon", "Judge Man", 
                        "Rogue Doll", "Kojikocy", "Uraby", "Beast of Talwar", "Mystic Horseman", "Terra the Terrible", "Dark Titan of Terror", 
                        "Master & Expert", 	"Mystic Clown", "Ogre of the Black Shadow", "Dark Energy", "Invigoration", "Dark Hole",
                        "Ookazi", "Ryu-Kishin Powered", "Swordstalker", "La Jinn the Mystical Genie of the Lamp", "Rude Kaiser", "Destroyer Golem",
                        "Skull Red Bird", "D. Human", "Pale Beast", "Fissure", "Trap Hole", "Mystical Space Typhoon", 	"De-Spell", "Castle Walls",
                        "Monster Reborn", "Reinforcements", "Raigeki", "Ultimate Offering", "Galaxy Serpent", "Dian Keto the Cure Master" ,
                        "Sakuretsu Armor", "Remove Trap"])


p1Deck = []
p2Deck = []
print()
print("Loading P1 Deck...")
print()
for name in p1DeckBuild:
  card = yugioh.get_card(card_name = name)
  p1Deck.append(card)

print()
print("Loading P2 Deck...")
print()
print()
for name in p2DeckBuild:
  card = yugioh.get_card(card_name = name)
  p2Deck.append(card)

print("Decks Loaded")


Loading P1 Deck...


Loading P2 Deck...


Decks Loaded


In [47]:
#Main
def letsDuel(p1Deck, p2Deck):
  print("LET'S DUEL!")
  print()
  #Field Structure
  # p1 spell/trap zone -> [None, None, None, None, None]  Field[0][0]
  # p1 monster zone -> [None, None, None, None, None]     Field[0][1]
  # p2 monster zone -> [None, None, None, None, None]     Field[1][0]
  # p2 spell/trap zone -> [None, None, None, None, None]  Field[1][1]
  Field = [[[], []], [[], []]]

  #grave for all used or destroyed cards
  p1GY = []
  p2GY = []

  #player
  p1 = 1
  p2 = 2   

  #Build and shuffle  decks
  useP1Deck = copy.deepcopy(p1Deck) 
  useP2Deck = copy.deepcopy(p2Deck) 
  rand.shuffle(useP1Deck)
  rand.shuffle(useP2Deck)
  #initialize Life Points
  p1LP = 4000
  p2LP = 4000
  print("Life Points Initialized")
  print("P1 LP =", p1LP)
  print("P2 LP =", p2LP)
  print()
  #initialize start hands
  p1Hand= []
  p2Hand= []
  print("Player 1 Draws Starting Hand")
  for i in range(5):
    draw(p1, useP1Deck, p1Hand, p1GY)
  print()
  print("Player 2 Draws Starting Hand")
  for i in range(5):
    draw(p2, useP2Deck, p2Hand, p2GY)
  print()
  #life points
  player = rand.randrange(1,3)

  print("Player ", player," Starts")
  global turn1 
  turn1 = True
  while (p1LP > 0) and (p2LP > 0) and (len(p1Deck) > 0) and (len(p2Deck) > 0):
    if player == 1:
      print()
      print()
      print("Player 1's Turn:")
      print()
      print("Draw Phase...")
      draw(player, useP1Deck, p1Hand, p1GY )
      print()
      print("Standby Phase...")
      p1LP, p2LP = strategy(player, p1Hand, Field, p1GY, p2GY, p1LP, p2LP)
      player = p2
    else:
      print()
      print()
      print("Player 2's Turn:")
      print()
      print("Draw Phase...")
      draw(player, useP2Deck, p2Hand, p2GY )
      print()
      print("Standby Phase...")
      p2LP, p1LP = strategy(player, p2Hand, Field, p2GY, p1GY, p2LP, p1LP)
      player = p1
    print()
    if p1LP > 0 and p2LP > 0:
      print("P1 LP =", p1LP)
      print("P2 LP =", p2LP)
    if (p1LP <= 0 and p2LP <= 0):
      print("P1 LP =", 0)
      print("P2 LP =", 0)
      print("Both Players LP Hit Zero: TIED GAME")
    elif p2LP <= 0:
      print("P1 LP =", p1LP)
      print("P2 LP =", 0)
      print("Player 2 LP Hit Zero: Player 1 Wins!")
    elif p1LP <= 0:
      print("P1 LP =", 0)
      print("P2 LP =", p2LP)
      print("Player 1 LP Hit Zero: Player 2 Wins!") 
    elif len(useP1Deck) == 0:
      print("P1 Deck Has", len(useP1Deck), "Cards")
      print("Since Player 1 Has Run Out Of Cards: Player 2 Wins")
    elif len(useP2Deck) == 0:
      print("P2 Deck Has", len(useP2Deck), "Cards")
      print("Since Player 2 Has Run Out Of Cards: Player 1 Wins")

  return p1LP, p2LP





In [106]:
#All Game Mechanics


#Summoning Mechanics
import array


def normalSummon(player, pHand, Field, monster):
   if player == 1:
      x = 0
      y = 1
   else:
      x = 1
      y = 0
   if len(Field[x][y]) < 5:
    for card in pHand:
      if card.name == monster.name:
        pHand.remove(card)
        newMon = onField(card, "attack", True)
        print("Summon", newMon)
        Field[x][y].append(newMon)
    

def flipSummon(player, Field, monsters):
  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0
  for i in range(len(Field[x][y])):
    for monster in monsters:
      if Field[x][y][i].name == monster.name:
        print("Flip Summon", Field[x][y][i])
        Field[x][y][i].pos = "attack"
        Field[x][y][i].isFaceUp = True
        
    
def setMonster(player, pHand, Field, monster):
    if player == 1:
      x = 0
      y = 1

    else:
      x = 1
      y = 0
     
    if len(Field[x][y]) < 5:
      for card in pHand:
        if card.name == monster.name:
          print("Set 1 Monster Face Down")
          newMon = onField(card, "defense", False)
          Field[x][y].append(newMon)
          pHand.remove(card)

def setTrap(player, pHand, Field, temp_traps):
    if player == 1:
      x = 0
      y = 0
    else:
      x = 1
      y = 1
    trap_counter = 0
    traps = []
    for trap in temp_traps:
        for card in pHand:
          if trap.name == card.name:
            traps.append(card)
    if Field[x][y]:
      for card in Field[x][y]:
        if "Trap" in card.type:
          trap_counter += 1
    set_counter = 3 - trap_counter
    if set_counter > 0:
      if set_counter > len(traps):
        print("Set", len(traps), "Trap(s)")
      else:
        print("Set", set_counter, "Trap(s)")
      delete = []
      for i in range(len(traps)): 
        if i < set_counter:
          newTrap = onField(traps[i], None, False)
          Field[x][y].append(newTrap)
          delete.append(traps[i])
      for card in delete:
        pHand.remove(card)

#Game Phases
def draw(player, deck, hand, GY):
  while len(hand) >= 7:
    minMon = None
    min = float('inf')
    counter = 0
    for card in hand: 
      if "Monster" in card.type:   
        counter += 1
        if card.attack < min:
          min = card.attack
          minMon = card
        elif card.attack == min and card.defense > minMon.defense:
          minMon = card
    if counter < 3:
      i = rand.randrange(len(hand))
      if "Monster" not in hand[i].type:
        discard = hand.pop(i)
        print("Too Many Cards In Hand: Discard", discard.name)
        GY.append(discard)
    else:
      print("Too Many Cards In Hand: Discard", minMon.name)
      hand.remove(minMon)
      GY.append(minMon)
  draw = deck.pop(0)
  print("Draw", draw.name)
  hand.append(draw)

def main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, main2_params, end_phase):
  print()
  print("Main Phase...")
  equipped_spells = main1_params["equipped_spells"]
  useful_spells = main1_params["useful_spells"]
  useful_traps = main1_params["useful_traps"]
  summon = main1_params["summon"]
  change_pos = main1_params["change_pos"]
  react = False

  if player == 1:
    x = 0
    y = 1
    notPlayer = 2
  else:
    x = 1
    y = 0
    notPlayer = 1



  #Activate useful spells
  for card in pHand:
      if card.name in useful_spells.keys() and useful_spells[card.name] == True:
          print("Activate", card.name)
          use, LP, opLP = activate(player,  Field, pHand, LP, opLP, GY, opGY, card, None)

  
  #Normal summon
  if summon != None and len(Field[x][y]) < 5:
    normalSummon(player,  pHand, Field, summon)

    for card in Field[y][y]:
      if "Trap Hole" == card.name:
        print("Opponent Activates", card.name)
        z = len(Field[x][y]) - 1
        use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, Field[x][y][z])
        react = use


  #Determine if strategy has to change
  if react:
    useful_spells, useful_traps, change_pos_def = counterStrategy(player, pHand, Field, GY, opGY, LP, opLP)
    main2_params["change_pos"] = change_pos_def
    change_pos = []
    #Clear opponents Monster Zone
    for card in pHand:
      if card.name == "Dark Hole" and useful_spells[card.name] == True:
          print("Activate", card.name)
          use, LP, opLP = activate(player, Field, pHand, LP, opLP, GY, opGY, card, None)

    #Activate useful spells
    for card in pHand:
        if card.name in useful_spells.keys() and useful_spells[card.name] == True:
            print("Activate", card.name)
            use, LP, opLP = activate(player,  Field, pHand, LP, opLP, GY, opGY, card, None)

  #Activate useful traps
  for card in Field[x][x]:
    if card.name in useful_traps.keys() and useful_traps[card.name] == True:
        print("Activate", card.name)
        use, LP, opLP = activate(player, Field, pHand, LP, opLP, GY, opGY, card, None)

  #Activate Equip Spells
  for card in pHand:
    if card.name in equipped_spells.keys() and equipped_spells[card.name] == True:
      print("Activate", card.name)
      use, LP, opLP = activate(player, Field, pHand, LP, opLP, GY, opGY, card, None)

  #Change position
  if change_pos:
      flip = []
      for monster in change_pos:
          if not monster.isFaceUp:
              flip.append(monster)
              change_pos.remove(monster)
      if change_pos:
        changePosition(player, Field, change_pos)
      if flip:
        flipSummon(player, Field, flip)

  #Battle phase
  LP, opLP = battle(player, Field, pHand, GY, opGY, LP, opLP, main2_params, end_phase)
  return LP, opLP
  
def battle(player, Field, pHand, GY, opGY, LP, opLP, main2_params, end_phase):
    print()
    print("Battle Phase...")
    global turn1
    monsters = []
    op_monsters = []
    counterTrap = False
    countered = False
    if turn1:
      turn1 = False
      LP, opLP = main2(player, Field, pHand, LP, opLP, GY, opGY, main2_params, end_phase)
      return LP, opLP

    if player == 1:
      x = 0
      y = 1
      notPlayer = 2
    else:
      x = 1
      y = 0
      notPlayer = 1

    Field[x][y].sort(reverse = True, key = atk_pts)
    Field[y][x].sort(reverse = True, key = atk_pts)
    for card in Field[y][y]:
      if "Waboku" == card.name:
        print("Opponent Activates", card.name)
        Field[y][y].remove(card)
        opGY.append(card)
        print("End Battle Phase")
        main2(player, Field, pHand, LP, opLP, GY, opGY, main2_params, end_phase)
        return LP, opLP
    
    i = 0
    while i < len(Field[x][y]):
      mon = Field[x][y][i]
      attacked = False
      i += 1
      if mon.can_atk and mon.pos == "attack":
        if Field[y][x]:
            j = 0
            while j < len(Field[y][x]):
              op = Field[y][x][j]
              j += 1
              if (not attacked):
                z = len(Field[y][x]) - 1
                if (op.pos== "attack") and (mon.total_atk > op.total_atk):
                    print(mon, "Attacks", op)
                    attacked = True
                    for card in Field[y][y]:
                      if not countered:
                        if "Sakuretsu Armor" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)
                          countered = True
                          counterTrap = True
                          i-=1
                          #attacked = False
                        elif "Magic Cylinder" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)
                          countered = True
                          counterTrap = True
                        elif "Reinforcements" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, None)
                          end_phase.append("Reinforcements")
                          countered = True
                    if counterTrap:
                      print("Attack Failed")
                      counterTrap = False
                      countered = False
                    else:
                      LP, opLP = attack(player, Field, GY, opGY, LP, opLP, mon, op)
                      countered = False
                   
                elif (op.pos== "defense") and (op.isFaceUp) and (mon.total_atk > op.total_def):
                    print(mon, "Attacks", op)
                    attacked = True
                    for card in Field[y][y]:
                      if not countered:
                        if "Sakuretsu Armor" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)
                          counterTrap = True
                          countered = True
                          i-=1
                          #attacked = False
                        elif "Magic Cylinder" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)
                          counterTrap = True
                          countered = True
                        elif "Castle Walls" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, None)
                          end_phase.append("Castle Walls")
                          countered = True
                    if counterTrap:
                      print("Attack Failed")
                      counterTrap = False
                      countered = False
                    else:
                      countered = False
                      LP, opLP = attack(player, Field, GY, opGY, LP, opLP, mon, op)
                    
                elif (op.isFaceUp == False):
                    print(mon, "Attacks", op)
                    attacked = True
                    for card in Field[y][y]:
                      if not countered:
                        if "Sakuretsu Armor" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)
                          counterTrap = True
                          i-=1
                          #attacked = False
                        elif "Magic Cylinder" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)
                          counterTrap = True
                        elif "Castle Walls" == card.name:
                          print("Activate", card.name)
                          use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, None)
                          end_phase.append("Castle Walls")
                          countered = True
                    if counterTrap:
                      print("Attack Failed")
                      counterTrap = False
                      countered = False
                    else:
                      LP, opLP = attack(player, Field, GY, opGY, LP, opLP, mon, op)
                      countered = False
                elif op == Field[y][x][z]:
                  attacked = True
                    
        else:
          print(mon, "Attacks Opponent Directly")
          attacked = True
          for card in Field[y][y]:
            if not counterTrap:
              if "Sakuretsu Armor" == card.name:
                print("Activate", card.name)
                use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)              
                counterTrap = True
                #attacked = False
              elif "Magic Cylinder" == card.name:
                print("Activate", card.name)
                use, opLP, LP = activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, mon)        
                counterTrap = True
          if counterTrap:
            print("Attack Failed")
            counterTrap = False
          else:
            LP, opLP = attack(player, Field, GY, opGY, LP, opLP, mon, None)
            counterTrap = False

          if opLP <= 0  or LP <= 0:
            return LP, opLP
             
      else:
        mon.can_atk = True

    LP, opLP = main2(player, Field, pHand, LP, opLP, GY, opGY, main2_params, end_phase)
    return LP, opLP

def main2(player, Field, pHand, LP, opLP, GY, opGY, main2_params, end_phase):
    print()
    print("Main Phase 2...")
    hand_traps = main2_params["hand_traps"]
    change_pos = main2_params["change_pos"]
    set_monster = main2_params["set"]
    useful_spells, useful_traps, change_pos = counterStrategy(player, pHand, Field, GY, opGY, LP, opLP)
    if player == 1:
      x = 0
      y = 1
      notPlayer = 2
    else:
      x = 1
      y = 0
      notPlayer = 1
    #Clear opponents Monster Zone
    for card in pHand:
        if card.name == "Dark Hole" and card.name in useful_spells.keys() and useful_spells[card.name] == True:
            use, LP, opLP = activate(player, Field, pHand, LP, opLP, GY, opGY, card, None)

    #summon an extra monster if possible
    for card in Field[x][x]:
      if card.name == "Ultimate Offering" and "Ultimate Offering" in useful_traps and useful_traps["Ultimate Offering"] == True:
        use, tempLP, tempOpLP, = activate(player, Field, pHand, LP, opLP, GY, opGY, card, None)
        for card in Field[y][y]:
          if "Trap Hole" == card.name:
            print("Opponent Activates Trap Hole")
            activate(notPlayer, Field, None, opLP, LP, opGY, GY, card, Field[x][y][len(Field[x][y]) - 1])
    #Activate useful spells
    for card in pHand:
        if card.name in useful_spells.keys() and useful_spells[card.name] == True:
            use, LP, opLP = activate(player, Field, pHand, LP, opLP, GY, opGY, card, None)


    if change_pos:
        changePosition(player, Field, change_pos)
    if hand_traps:
        setTrap(player, pHand, Field, hand_traps)
    if set_monster != None:
          setMonster(player, pHand, Field, set_monster)


    end(player, Field, end_phase)
    return LP, opLP
 
def end(player, Field, end_phase):
    print()
    print("End Phase...")
    
    if player == 1:
        x = 0
        y = 1
    elif player == 2:
        x = 0
        y = 1

    #Reset each card's ability to change position
    for card in Field[x][y]:
        card.posChanged = False
    # Check if end_phase is empty
    if end_phase:
        #Check if end_phase contain Reinforcement or Castle Walls or Change of Hearts before checking each card(Probably Dont need to Check)
        if any(card == "Reinforcements" or card == "Castle Walls" or card == "Change of Heart" for card in end_phase):
            #Go to each card in player field
            for card in Field[x][y]:
                if card.equipped:
                    for equipped_card in card.equipped:
                        if equipped_card == "Reinforcements":
                            print("Reinforcements Effect Ends: " + card.name + " Loses 500 Atk")
                            card.equipped.remove(equipped_card)
                            card.increaseBonusAtk(-500)
                        if equipped_card == "Castle Walls":
                            print("Castle Walls Effect Ends: " + card.name + " Loses 500 Def")
                            card.equipped.remove(equipped_card)
                            card.increaseBonusDef(-500)
                #Change of Heart Implementation
                if card.switched:
                    #Send Opponent back to their Field
                    print("Change of Heart Effect Ends: " + card.name + " Goes Back to Opponents Field")
                    Field[y][x].append(card)
                    Field[x][y].remove(card)
                    card.switched = False
     
    print("End Turn")
    
    

#Other Game mechanics
def changePosition(player, Field, monsters):
  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0
  if monsters[0].pos == 'attack':
    print("Switching All Monsters To Defense Position")
  for i in range(len(Field[x][y])):
    for monster in monsters:
      if Field[x][y][i].name == monster.name and Field[x][y][i].posChanged == False:
        Field[x][y][i].posChanged = True
        if Field[x][y][i].pos == "attack":
          print( monster, "Switched to Defense Position")
          Field[x][y][i].pos = 'defense'
        else:
          print(monster, "Switched to Attack Position")
          Field[x][y][i].pos = "attack"
         
   

def attack(player, Field, GY, opGY, LP, opLP, attacking, attacked):
  if player == 1:
    x = 0
    y = 1
    notPlayer = 2
  else:
    x = 1
    y = 0
    notPlayer = 1
  if attacked == None:
    print("Attack Successful")
    print("Opponent loses", attacking.total_atk, "Life Points")
    opLP -= attacking.total_atk
  elif attacked.pos == "attack":
    lpLoss = attacking.total_atk - attacked.total_atk
    if lpLoss > 0:
      print("Attack Successful")
      print(attacked.name, "Destroyed")
      print("Opponent Loses", lpLoss, "Life Points" )
      opLP -= (lpLoss)
      destroy(notPlayer, Field, opGY, attacked)
      
    elif lpLoss < 0:
      print("Attack Failed")
      print(attacking.name, "Destroyed")
      print("I Lose ", -lpLoss, "Life Points" )
      LP += (lpLoss)
      destroy(player, Field, GY, attacking)
    else:
      print("Attack Semi-Successful")
      print("Both Monsters Destroyed")
      destroy(player, Field, GY, attacking)
      destroy(notPlayer, Field, opGY, attacked)
  else:
    if not attacked.isFaceUp:
      attacked.isFaceUp = True
    if attacked.total_def >= attacking.total_atk:
      lpLoss = attacked.total_def - attacking.total_atk
      print("Attack Failed")
      print("I Lose", lpLoss, "Life Points")
      LP -= (lpLoss)
    else:
      print("Attack Successful")
      print(attacked.name, "Destroyed")
      destroy(notPlayer, Field, opGY, attacked)
  
  return LP, opLP

def destroy(player, Field, GY, monster):
  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0
  for destroyed in Field[x][y]:
    if monster.name == destroyed.name:
      Field[x][y].remove(destroyed)
      GY.append(destroyed)
      destroyed.bonus_atk = 0
      destroyed.bonus_def = 0
      destroyed.total_atk = destroyed.attack
      destroyed.total_def = destroyed.defense
      if destroyed.equipped:
          for card in Field[x][x]:
            if card.name in destroyed.equipped:
              Field[x][x].remove(card) 
          destroyed.equipped = []
  

def activate(player, Field, pHand, pLP, opLP, GY, opGY, card, opMonster):
  used = False
  cardName = card.name
  equip_spells = ["Dark Energy", "Invigoration", "Sword of Dark Destruction", "Book of Secret Arts"]
  if player == 1:
    x = 0
  else:
    x = 1
  if 'trap' in card.type.lower():
    if cardName == "Castle Walls":
      used = CastleWalls(player,Field)
    elif cardName == "Sakuretsu Armor":
      used = SakuretsuArmor(player, Field, opGY, opMonster)
    elif cardName == "Trap Hole":
      used = trapHole(player, Field, opGY, opMonster)
    elif cardName == "Reinforcements":
      used = reinforcements(player, Field)
    elif cardName == "Waboku":
      used = waboku(player, Field)
    elif cardName == "Magic Cylinder":
      used, opLP = magicCylinder(player, opMonster, opLP)
    elif cardName == "Ultimate Offering":
      used, pLP = ultimateOffering(player, pHand, Field, pLP)
    else:
      print("Trap Card Not Found")

    if used:
      for trap in Field[x][x]:
        if trap.name == cardName:
          Field[x][x].remove(trap)
          GY.append(trap)
  elif "spell" in card.type.lower() and len(Field[x][x]) < 5:
    if cardName == "Dark Energy": 
      used = DarkEnergy(player, Field)
    elif cardName == "Invigoration":
      used = Invigoration(player, Field)
    elif cardName == "Sword of Dark Destruction":
      used = SwordOfDarkDestruction(player, Field)
    elif cardName == "Book of Secret Arts":
      used = BookOfSecretArts(player, Field)
    elif cardName == "Change of Heart":
      used = changeOfHeart(player, Field)
    elif cardName == "Fissure":
      used = fissure(player, Field, opGY)
    elif cardName == "Ookazi":
      used, opLP = ookazi(opLP)
    elif cardName == "Mystical Space Typhoon":
      used = mysticalSpaceTyphoon(player, Field, opGY)
    elif cardName == "De-Spell":
      used = deSpell(player, Field, opGY)
    elif cardName == "Monster Reborn":
      used = monsterReborn(player, Field, GY, opGY)
    elif cardName == "Remove Trap":
      used = removeTrap(player, Field, opGY)
    elif cardName == "Raigeki":
      used = raigeki(player, Field, opGY)
    elif cardName == "Dian Keto the Cure Master":
      used, pLP = DianKetotheCureMaster(pLP)
    elif cardName == 'Dark Hole':
      used = DarkHole(player, Field, GY, opGY)
    else:
      print("Spell Card Not Found")
    
    if cardName in equip_spells and used:
      for spell in pHand:
        if spell.name == cardName:
          pHand.remove(spell)
          activatedCard = onField(spell, None, True)
          Field[x][x].append(activatedCard)
    elif used:
       for spell in pHand:
        if spell.name == cardName:
          pHand.remove(spell)
          GY.append(spell)
  else:
    print("Cannot Use This Spell/Trap")
  return used, pLP, opLP



def strategy(player, pHand, Field, GY, opGY, LP, opLP):
  print("Strategizing. Please Wait...")
  print("Strategy (")
  temp_field = copy.deepcopy(Field[:][:][:])
  temp_hand = copy.deepcopy(pHand)
  tempLP = LP
  tempOpLP = opLP
  tempGY = copy.deepcopy(GY)
  tempOpGY = copy.deepcopy(opGY)
  hand_monsters = []
  monsters = []
  change_pos = []
  useful_spells = {}
  equipped_spells = {}
  useful_traps = {}
  hand_traps = []
  end_phase = []
  field_test = False
  optimal_monster = None
  op_optimal_monster = None
  min_monster = None
  equip_spells = [ "Invigoration", "Sword of Dark Destruction", "Book of Secret Arts", "Dark Energy"]

  if player == 1:
    x = 0
    y = 1
    notPlayer = 2
  else:
    x = 1
    y = 0
    notPlayer = 1

 
  if Field[x][y]:
    for card in pHand:
      if "Raigeki" == card.name:
        pHand.remove(card)
        pHand.insert(0, card)
        temp_hand = copy.deepcopy(pHand)
  for card in pHand:
    if "Dark Hole" == card.name and pHand[0].name != "Raigeki":
        pHand.remove(card)
        pHand.insert(0, card)
        temp_hand = copy.deepcopy(pHand)
    elif "Dark Hole" == card.name:
        pHand.remove(card)
        pHand.insert(1, card)
        temp_hand = copy.deepcopy(pHand)

  counter = 0
  for card in temp_hand:
    if 'monster' in card.type.lower():
      hand_monsters.append(card)
    if 'spell' in card.type.lower() and not (card.name in equip_spells):
      print("Activate", card.name)
      use, tempLP, tempOpLP, = activate(player, temp_field, temp_hand, tempLP, tempOpLP, tempGY, tempOpGY, card, None)
      useful_spells[card.name] = use
    if 'trap' in card.type.lower():
      hand_traps.append(card)
      counter += 1

    for card in temp_field[x][x]:
      if "Ultimate Offering" == card.name and len(hand_monsters) > 1:
        print("Activate", card.name)
        use, tempLP, tempOpLP, = activate(player, temp_field, temp_hand, tempLP, tempOpLP, tempGY, tempOpGY, card, None)
        useful_traps[card.name] = use

  if counter > 0:
    print("Set Trap(s)")
  

    
  if "Change of Heart" in useful_spells.keys() and useful_spells["Change of Heart"] == True:
    end_phase.append("Change of Heart")
  if temp_field[x][y] or hand_monsters:
    if temp_field[x][y]:
      optimal_monster_on_field = max(temp_field[x][y], key=lambda monster: monster.total_atk)
      on_field = optimal_monster_on_field.total_atk
    else:
      optimal_monster_on_field = None
      on_field = float('-inf')
    if hand_monsters:
      optimal_monster_in_hand = max(hand_monsters, key = lambda monster: monster.attack)
      in_hand = optimal_monster_in_hand.attack
    else:
      optimal_monster_in_hand = None
      in_hand = float('-inf')
     
    if on_field < in_hand and len(temp_field[x][y]) < 5:
      print("Try Summoning Best Monster")
      normalSummon(player, temp_hand, temp_field, optimal_monster_in_hand)
      z = len(temp_field[x][y])-1
      optimal_monster =  temp_field[x][y][z]
      field_test = True 
    else:
      optimal_monster = optimal_monster_on_field

 

  if optimal_monster != None:
    for card in temp_hand:
      if (card.name in equip_spells):
        use, tempLP, tempOpLP, = activate(player, temp_field, temp_hand, tempLP, tempOpLP, tempGY, tempOpGY, card, None)
        equipped_spells[card.name] = use #equip spell to optimal monster on temp_field


    if temp_field[y][x]:
      op_optimal_monster = max(temp_field[y][x], key=lambda monster: monster.total_atk)


    if temp_field[x][y]:
      optimal_monster = max(temp_field[x][y], key=lambda monster: monster.total_atk)
    if op_optimal_monster != None:
      if (op_optimal_monster.pos == 'attack') and (optimal_monster.total_atk + 500 > op_optimal_monster.total_atk):
        for card in temp_field[x][x]:
          if "Reinforcements" == card.name:
            use, tempLP, tempOpLP, = activate(player, temp_field, temp_hand, tempLP, tempOpLP, tempGY, tempOpGY, card, None)
        for card in hand_traps: 
          if "Reinforcements" == card.name:
            optimal_monster.increaseBonusAtk(500)
            hand_traps.remove(card)
            hand_traps.insert(0, card)
      elif (op_optimal_monster == 'defense' and op_optimal_monster.total_def < optimal_monster.total_atk) or (op_optimal_monster == 'attack'):
        pass
      else:
        for card in temp_field[x][x]:
          if "Reinforcements" == card.name:
            print("Activate", card.name)
            use, tempLP, tempOpLP, = activate(player, temp_field, temp_hand, tempLP, tempOpLP, tempGY, tempOpGY, card, None)
            useful_traps[card.name] = use
            if use:
              end_phase.append("Reinforcements")

      if optimal_monster_in_hand != None:
        min_monster = min(hand_monsters, key = lambda monster: monster.attack)
      monsters = []
      for monster in temp_field[x][y]:
        if monster.pos == "attack":
          monsters.append(monster)
      monsters.sort(reverse = True, key = atk_pts)
      

      op_monsters_atk = []
      for monster in temp_field[y][x]:
        if monster.pos == "attack":
          op_monsters_atk.append(monster)
      op_monsters_atk.sort(reverse = True, key = atk_pts)

      op_monsters_def = []
      for monster in temp_field[y][x]:
        if monster.pos == "defense" and monster.isFaceUp == True:
          op_monsters_def.append(monster)
      op_monsters_def.sort(reverse = True, key = def_pts)
      for monster in temp_field[y][x]:
        if monster.pos == "defense" and monster.isFaceUp == False:
          op_monsters_def.append(monster)

      op_monsters = op_monsters_atk + op_monsters_def

      if op_optimal_monster.total_atk >= optimal_monster.total_atk:
        
        main1_params = {"equipped_spells": {}, "useful_spells": useful_spells, "useful_traps": useful_traps, "summon": None, "change_pos": [] }
        #battle_params = {"monsters": monsters, "op_monsters": op_monsters}
        main2_params = {"hand_traps": hand_traps, "change_pos": monsters, "set": min_monster}
        print(') Strategy Found: Opponent Has Advantage, Play Defense')
        main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, main2_params, end_phase)
        

      else: 
        l = len(temp_field[y][y])
        danger = False
        if l >= 2:
        
          if LP < 1000:
            main1_params = {"equipped_spells": {}, "useful_spells": useful_spells, "useful_traps": useful_traps, "summon": None, "change_pos": [] }
            #battle_params = {"monsters": monsters[l-2:], "op_monsters": op_monsters}
            main2_params = {"hand_traps": hand_traps, "change_pos": monsters, "set": min_monster}
            print(') Strategy Found: Opponent Has Advantage, Play Defense')
            main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, main2_params, end_phase)

          elif hand_monsters:
            hand_monsters2 = []
            for card in temp_hand:
              if "Monster" in card.type:
                hand_monsters2.append(card)
            if hand_monsters2:
              optimal_monster_in_hand2 = max(hand_monsters2, key = lambda monster: monster.attack)
              optimal_points2 = max(optimal_monster_in_hand2.attack, on_field)
              if (optimal_points2 > op_optimal_monster.total_atk) and field_test:
                temp_hand.append(temp_field[x][y].pop())
                print("Opponent Has Too Many Spell/Traps. Save Best Monster. Try Second Best.")
                normalSummon(player, temp_hand, temp_field, optimal_monster_in_hand2)

        attack_wins = [False for i in range(len(op_monsters))]
        i = 0
        if field_test:
          z = len(temp_field[x][y]) - 1
          monsters.append(temp_field[x][y][z])
        for monster in monsters:
          if i < len(op_monsters):
            op = op_monsters[i]
            if op.pos == "attack": 
              if monster.total_atk > op.total_atk:
                attack_wins.append(True)
              else:
                attack_wins.append(False)
            else:
              if monster.total_atk > op.total_def:
                attack_wins.append(True)
              else:
                attack_wins.append(False)
            i = i + 1

        monsters_def = []
        for monster in temp_field[x][y]:
          if monster.pos == "defense":
            monsters_def.append(monster)
        monsters_def.sort(reverse = True, key = atk_pts)
        i = 0
        change_pos = []
        for monster in monsters_def:
          if i < len(op_monsters):
            op = op_monsters[i]
            win = attack_wins[i]
            if (not win) and op.total_atk < monster.total_atk:
              change_pos.append(monster)
              monsters.append(monster)
            i += 1
         

        monsters.sort(reverse = True, key = atk_pts)


          
        main1_params = {"equipped_spells": equipped_spells, "useful_spells": useful_spells, "useful_traps": useful_traps, "summon": optimal_monster_in_hand, "change_pos": change_pos }
        #battle_params = {"monsters": monsters, "op_monsters": op_monsters}
        main2_params = {"hand_traps": hand_traps, "change_pos": [], "set": None}
        print(') Strategy Found: Advantage is Mine, Play Offense')
        LP, opLP = main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, main2_params, end_phase )
    else:
      if LP > 1000:
        main1_params = {"equipped_spells": equipped_spells, "useful_spells": useful_spells, "useful_traps": useful_traps, "summon": optimal_monster_in_hand, "change_pos": change_pos }
        #battle_params = {"monsters": monsters, "op_monsters": []}
        main2_params = {"hand_traps": hand_traps, "change_pos": [], "set": None}
        print(') Strategy Found: Advantage is Mine, Play Offense')
        LP, opLP = main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, main2_params, end_phase )
      else:
        main1_params = {"equipped_spells": {}, "useful_spells": useful_spells, "useful_traps": useful_traps, "summon": optimal_monster_in_hand, "change_pos": [] }
        #battle_params = {"monsters": monsters, "op_monsters": op_monsters}
        main2_params = {"hand_traps": hand_traps, "change_pos": monsters, "set": None}
        print(') Strategy Found: Advantage is Mine, Play Offense')
        main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, main2_params, end_phase)
  else:
    main1_params = {"equipped_spells": {}, "useful_spells": useful_spells, "useful_traps": useful_traps, "summon": None, "change_pos": [] }
    #battle_params = {"monsters": [], "op_monsters": []}
    main2_params = {"hand_traps": hand_traps, "change_pos": [], "set": None}
    print(') Strategy Found: No Monsters. Rely On Spell/Traps')
    main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, main2_params, end_phase)
  return LP, opLP
   
def counterStrategy(player, pHand, Field, GY, opGY, LP, opLP):
  useful_spells = {}
  useful_traps = {}
  change_pos = []
  hand_traps = []
  temp_hand = copy.deepcopy(pHand)
  temp_field = copy.deepcopy(Field[:][:][:])
  tempGY = copy.deepcopy(GY)
  tempOpGY = copy.deepcopy(opGY)
  tempLP = LP
  tempOpLP = opLP
  equip_spells = [ "Invigoration", "Sword of Dark Destruction", "Book of Secret Arts", "Dark Energy"]

  print("Updating Strategy... (")
  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0
  
  for card in pHand:
    if "Dark Hole" == card.name:
        pHand.remove(card)
        pHand.insert(0, card)
        temp_hand = copy.deepcopy(pHand)


  for card in temp_hand:
    if 'spell' in card.type.lower() and not (card.name in equip_spells):
      print("Activate", card.name)
      use, tempLP, tempOpLP, = activate(player, temp_field, temp_hand, tempLP, tempOpLP, tempGY, tempOpGY, card, None)
      useful_spells[card.name] = use
    if 'trap' in card.type.lower():
      hand_traps.append(card)

  for card in Field[x][x]:
    if "Ultimate Offering" == card.name:
      print("Activate", card.name)
      use, tempLP, tempOpLP, = activate(player, temp_field, temp_hand, tempLP, tempOpLP, tempGY, tempOpGY, card, None)
      useful_traps[card.name] = use

  if Field[x][y]:
    optimal_monster = max(Field[x][y], key = lambda monster: monster.total_atk)
  else: 
    optimal_monster = None
  if Field[y][x]:
    op_optimal_monster = max(Field[y][x], key = lambda monster: monster.total_atk)
  else:
    op_optimal_monster = None
  if (optimal_monster != None and op_optimal_monster != None and optimal_monster.total_atk < op_optimal_monster.total_atk) or (LP < 1000):
    print("Opponents Advantage. Play Defense.")
    for monster in Field[x][y]:
      if monster.pos == "attack":
        change_pos.append(monster)
  print(")...Strategy Updated")
  return useful_spells, useful_traps, change_pos

In [14]:
#Card Effects

# examples = 'Monster Reborn', 'Remove Trap', 'Raigeki' 
#(cards return None if the activation cost can't be met, or they are useless)

#cards that still need to be defined
#set 1: Sword of Dark Destruction', 'Book of Secret Arts', 'Dark Hole', 'Dian Keto the Cure Master' 
#set 2: 'Fissure', 'Trap Hole', 'De-Spell', 'Reinforcements'
#set 3: 'Ultimate Offering', 'Ookazi', 'Mystical Space Typhoon', 'Magic Cylinder'
#set 4: 'Castle Walls', 'Invigoration', 'Sakuretsu Armor', 'Dark Energy'
#set 5: 'Change of Heart', 'Waboku', 'Dragon Capture Jar'

#Mahdsiah's Cards
def magicCylinder(player, opAttackingCard, opLP):
  print("Reflect Opponents Attack")
  print("Opponent loses", opAttackingCard.total_atk, "Life Points")
  opLP -= opAttackingCard.total_atk
  return True, opLP

def ookazi(opLP):
  print("Opponent's Life Points decreased by 800")
  opLP -= 800
  return True, opLP

def mysticalSpaceTyphoon(player, Field, opGY):
  if player == 2:
    x = 0
    y = 1
  else:
    x = 1
    y = 0
  
  if len(Field[x][x]) > 0:
    print(Field[x][x][0].name, "destroyed" )
    opGY.append(Field[x][x][0])
    card = Field[x][x].pop(0)
    for monster in Field[x][y]:
      if card.name in monster.equipped:
        if card.name == "Invigoration" or card.name == "Sword of Dark Destruction":
          print(monster.name, "Loses 400 atk and Gains 200 def")
          monster.increaseBonusAtk(-400)
          monster.increaseBonusDef(200) 
        elif card.name == "Book of Secret Arts":
          print(monster.name, "Loses 300 atk and Gains 300 def")
          monster.increaseBonusAtk(-300)
          monster.increaseBonusDef(300)
        elif card.name == "Dark Energy":
          print(monster.name, "Loses 300 atk and def")
          monster.increaseBonusAtk(-300)
          monster.increaseBonusDef(-300)
    return True
  else:
    print("Opponent has no spell/traps to destroy")
    return False


def ultimateOffering(player, pHand, Field, pLP):
  max = 0
  maxMon = None
  min = float('inf')
  minMon = None
  if player == 1:
    x = 0
    y = 1
  else: 
    x = 1
    y = 0


  if len(Field[x][y]) <= 5 and len(pHand) > 0 and pLP > 500:
    for monster in pHand:
      if "Monster" in monster.type:
        if max <= monster.attack:
          max = monster.attack
          maxMon = monster
        if min > monster.attack:
          min = monster.attack
          minMon = monster
    if maxMon == None and minMon == None:
      print("No monsters to summon")
      return False, pLP
    max2 = max
    for monster in Field[x][y]:
      if monster.total_atk> max2:
        max2 = monster.total_atk
    if len(Field[y][x]) > 0:
      for monster in Field[y][x]:
        if max2 < monster.total_atk:
          print("Pay 500 Life Points to Set", minMon.name)
          pLP = pLP - 500
          setMonster(player, pHand, Field, minMon)
          return True, pLP
    print("Pay 500 Life Points to Summon", maxMon.name)
    pLP = pLP - 500
    normalSummon(player, pHand, Field, maxMon)
    return True, pLP
  else:
    print("Not Enough Life Points to Use")
    return False, pLP

#special someone monster from either GY
def monsterReborn(player, Field, GY, opGY):
  maxNum = float('-inf')
  maxMonster = None

  #initialize player field
  if player == 1:
    x = 0
    y = 1
  elif player == 2:
    x = 1
    y = 0
  
  #if either Graveyard has cards and your field isnt full
  if len(Field[x][y]) <= 5:
    op = False


    if len(GY) == 0 and len(opGY) ==0:
      print("No Monsters To Special Summon")
      return False

    # search for best monster in either graveyard
    if len(GY) > 0:
      for card in GY:
          cardType = re.search("monster", card.type.lower())
          if (cardType != None) and (card.attack > maxNum):
            maxNum = card.attack
            maxMonster = card
          elif (cardType != None) and card.attack == maxNum and card.defense > maxMonster.defense:
            maxNum = card.attack
            maxMonster = card
    if len(opGY) > 0:
      for card in opGY:
          cardType = re.search("monster", card.type.lower())
          if (cardType != None) and (card.attack > maxNum):
            maxNum = card.attack
            maxMonster = card
            op = True
          elif (cardType != None) and card.attack == maxNum and card.defense > maxMonster.defense:
            maxNum = card.attack
            maxMonster = card
            op = True
    if maxMonster != None and op == False:
      GY.remove(maxMonster)
    elif maxMonster != None and op == True:
      opGY.remove(maxMonster)

  #if you can't summon a monster return None
    else:
      print("No Monsters to Special Summon")
      return False

    #if there exists such a monster determine if it is stronger than opponents monsters
    pos = 'attack'
    if Field[y][x]:
      opMax = max(Field[y][x], key = lambda monster: monster.total_atk)
      if opMax.total_atk > maxMonster.attack:
        pos = 'defense'
      
    summon = onField(maxMonster, pos, True)
    print("Special Summon", summon, "From the Grave")
    Field[x][y].append(summon)
    return True
    
  else:
    print("No Monsters to Special Summon")
    return False

def removeTrap(player, Field, opGY):
  if player == 1:

    #set (x, y) values to opposite oppenents spell/trap zone
    x = y = 1

  elif player == 2:

    #set (x, y) values to opposite oppenents spell/trap zone
    x = y = 0 

  trap = None

  #if opponent has a spell or trap on the field
  if len(Field[x][y]) > 0:

    #search for a trap to destroy
    for card in Field[x][y]:
        cardType = re.search("trap", card.type.lower())
        if (cardType != None):
          trap = card
          break
    
    #destroy trap
    if trap == None:
      print("No traps to destroy")
      return False
    opGY.append(trap)
    Field[x][y].remove(trap)
    print(trap.name, "destroyed")
    return True
  else:
    print("No cards in spell/trap zone")
    #return None if no traps can be destroyed
    return False

def raigeki(player, Field, opGY):
  
  #set field (x, y) values to opposite players field
  if player == 1:
    x = 1
    y = 0
    notPlayer = 2
  else:
    x = 0
    y = 1
    notPlayer = 1

  if len(Field[x][y]) > 0:
    
    #destroy each monster on the opponents field
    for z in range(len(Field[x][y])-1, -1, -1):
      print("Opponents", Field[x][y][z].name,"Destroyed")
      destroy(notPlayer, Field, opGY, Field[x][y][z])
    return True 
    
  #if opponent has no monsters on their field
  else:
    print("Opponent Has No Monsters To Destroy")
    return False



In [7]:
#Calvin's Cards

def SwordOfDarkDestruction(player, Field):
    # Equip only to a DARK monster. It gains 400 ATK and loses 200 DEF.

    # Most Optimal is to equip it to Player. The opponent side doesn't matter since 
    # increase the attack on player will outweight of decreasing defense in opponent side
    
    AttackIncrease = 400
    DefenseDecrease = -200   
    
    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    dark_monsters = []
    for monster in Field[x][y]:
        if monster.attribute == "DARK":
            dark_monsters.append(monster)
    if not (dark_monsters):
        print("No DARK monsters on the field to equip the 'Sword of Dark Destruction' to!")
        return False
    else: 
        optimal_monster = max(dark_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Sword of Dark Destruction' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.increaseBonusAtk(AttackIncrease)
        optimal_monster.increaseBonusDef(DefenseDecrease)
        optimal_monster.equipped.append('Sword of Dark Destruction')
        return True
     


def BookOfSecretArts(player, Field):
   # A Spellcaster-Type monster equipped with this card increases its ATK and DEF by 300 points

    # Most Optimal is to equip it to Player monster 
    AttackIncrease = 300
    DefenseDecrease = -300   

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    spellcaster_monsters = []
    for monster in Field[x][y]:
        if monster.race == "Spellcaster":
            spellcaster_monsters.append(monster)

    if not (spellcaster_monsters):
        print("No SpellCaster monsters on the field to equip the 'Book Of Secret Arts' to!")
        return False
    else: 
        optimal_monster = max(spellcaster_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Book Of Secret Arts' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.increaseBonusAtk(AttackIncrease)
        optimal_monster.increaseBonusDef(DefenseDecrease)
        optimal_monster.equipped.append('Book of Secret Arts' )
        return True

def DarkHole(player, Field, GY, opGY):
    # Destroy all monsters on the field.
    if player == 1:
      x = 0
      y = 1
      notPlayer = 2
    else:
      x = 1
      y = 0
      notPlayer = 1

    monsterField1 = Field[x][y]
    monsterField2 = Field[y][x]

    # Return None if nothin on opponents Field
    if len(monsterField2) == 0:
        print("Opponent Has No Monsters to Destroy")
        return False

    #if opponent controls monster and player doesn't, activate destroy opponents monsters
    elif len(monsterField1) == 0:
        for z in range(len(monsterField2)-1, -1, -1):
            destroy(notPlayer, Field, opGY, monsterField2[z])
        return True
    #if both player and opponent controls monster see who has strongest monster
    else:
      optimal_monster = max(monsterField1, key=lambda monster: monster.total_atk)
      op_optimal_monster = max(monsterField2, key=lambda monster: monster.total_atk)

      #if opponent has the strongest monster send all monsters to the GY
      if optimal_monster.total_atk < op_optimal_monster.total_atk:
        for z in range(len(monsterField1)-1, -1, -1):
            destroy(player, Field, GY, monsterField1[z])
        for z in range(len(monsterField2)-1, -1, -1):
            destroy(notPlayer, Field, opGY, monsterField2[z])
        return True
      #else don't use this card
      else:
        print("I Have The Best Monster. Save Card For Later.")
        return False
       
 

  
    
def DianKetotheCureMaster(LP):
    # Increase your Life Points by 1000 points.
    LP += 1000
    return True, LP

In [8]:
#Chase's Cards

def CastleWalls(player,Field):
    #Increase a selected monster's DEF by 500 points during the turn this card is activated.
    DefenseIncrease = 500

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    #Finds highest defense monster
    if Field[x][y]:
      monsters = Field[x][y]
      optimal_monster = max(monsters, key=lambda monster: monster.total_def)
      tempAttribute = optimal_monster.total_def + DefenseIncrease
      #switch player field
      temp = x
      x = y
      y = temp
    else:
      print("No Monster to Equip To")
      return False

    #finds opponents highest attack monster
    if Field[y][x]:
      monsters = Field[y][x]
      op_optimal_monster = max(monsters, key=lambda monster: monster.total_atk)
    else:
      return False
    #if oppenent highest attack monster points is higher than players highest defense monster points then it is useless
    if op_optimal_monster.total_atk > tempAttribute:
        print("Opponent Will Destroy My Monster Anyway. Save For Later.")
        return False
    else:
        print("Equipping 'Castle Walls' to", optimal_monster.name, "to increase its defense by", DefenseIncrease)
        optimal_monster.increaseBonusDef(DefenseIncrease)
        optimal_monster.equipped.append('Castle Walls')
        return True
        

def DarkEnergy(player, Field):
    #A Fiend-Type monster equipped with this card increases its ATK and DEF by 300 points.
    AttackIncrease = 300
    DefenseIncrease = 300 

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    fiend_monsters = []
    for monster in Field[x][y]:
        if monster.race == "Fiend":
            fiend_monsters.append(monster)

    if not (fiend_monsters):
        print("No FIEND-TYPE monsters on the field to equip the 'Dark Energy' to!")
        return False
    else: 
        optimal_monster = max(fiend_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Dark Energy' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and increase its defense by", DefenseIncrease)
        optimal_monster.increaseBonusAtk(AttackIncrease)
        optimal_monster.increaseBonusDef(DefenseIncrease)
        optimal_monster.equipped.append('Dark Energy')
        return True



def Invigoration(player, Field):
    #An EARTH monster equipped with this card increases its ATK by 400 points and decreases its DEF by 200 points
    AttackIncrease = 400
    DefenseDecrease = -200   

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    earth_monsters = []
    for monster in Field[x][y]:
        if monster.attribute == "EARTH":
            earth_monsters.append(monster)

    if not (earth_monsters):
            print("No EARTH monsters on the field to equip the 'Invigoration to!")
            return False
    else: 
        optimal_monster = max(earth_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Invigoration' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.increaseBonusAtk(AttackIncrease)
        optimal_monster.increaseBonusDef(DefenseDecrease)
        optimal_monster.equipped.append('Invigoration')
        return True

def SakuretsuArmor(player, Field, opGY, opAttackingCard):
    #When an opponent's monster declares an attack: Target the attacking monster and destroy it

    #only can be used when opponet is attacking, so return none if op is not
    if opAttackingCard == None:
        print("No Attacking Monster")
        return False

    else:
      # set field (x, y) values to opposite players field
        if player == 1:
            x = 1
            y = 0
            notPlayer = 2
        else:
            x = 0
            y = 1
            notPlayer = 1
        op_Field = Field[x][y]
        if len(op_Field) > 0:
            
            #destroy monster on the opponents field
            count=0
            mon = opAttackingCard
            destroy(notPlayer, Field, opGY, mon)
            print(mon.name,"Destroyed")
            return True
            
        #if opponent has no monsters on their field
        else:
            print("Opponent Has No Monsters")
            return False


In [109]:
#Matt's Cards

def fissure(player, Field, opGY):
  # Destroy the one face up monster that has the lowest attack.

  if player == 1:
    x = 1
    y = 0
    notPlayer = 2
  else:
    x = 0
    y = 1
    notPlayer = 1

  if len(Field[x][y]) > 0:
    min_atk_monster = min(Field[x][y], key=lambda monster: monster.total_atk)
    destroy(notPlayer, Field, opGY, min_atk_monster)
    print(min_atk_monster.name + " Destroyed")
    return True

  else:
    print("Opponent Controls No Monsters.")
    return False


def trapHole(player, Field, opGY, summonedMonster):
  # Destroys target monster with 1000 or more ATK.

  if player == 1:
    notPlayer = 2
  else:
    notPlayer = 1

  monster = summonedMonster
  if monster.attack >= 1000:
    destroy(notPlayer, Field, opGY, monster)
    print(monster.name + " Destroyed")
    return True
  else:
    print("Opponents Monster Doesn't Meet Requirement For Destruction.")
    return False


def deSpell(player, Field, opGY):
  # Target card on field; destroy that card if it is a spell card.
  found = False
  if player == 1:
    x = 1
    y = 0
  else:
    x = 0
    y = 1

  if len(Field[x][x]) > 0:

    for card in Field[x][x]:
      cardType = re.search("spell", card.type.lower())
      if cardType is not None:
        found = True
        spell = card
        opGY.append(spell)
        i = Field[x][x].index(spell)
        Field[x][x].pop(i)
        print(spell.name + " removed from the opponent's spell and trap zone.")
        for monster in Field[x][y]:
          if spell.name in monster.equipped:
            if spell.name == "Invigoration" or spell.name == "Sword of Dark Destruction":
              print(monster.name, "Loses 400 atk and Gains 200 def")
              monster.increaseBonusAtk(-400)
              monster.increaseBonusDef(200) 
            elif spell.name == "Book of Secret Arts":
              print(monster.name, "Loses 300 atk and Gains 300 def")
              monster.increaseBonusAtk(-300)
              monster.increaseBonusDef(300)
            elif spell.name == "Dark Energy":
              print(monster.name, "Loses 300 atk and def")
              monster.increaseBonusAtk(-300)
              monster.increaseBonusDef(-300)
        return True
      if not found:
        print("Opponent Has No Spells")
        return False
    else:
      print("Opponent has no spell cards.")
      return False


def reinforcements(player, Field):
  # Targets 1 face-up monster on the field. Adds 500 ATK until the end of the turn.

  AttackIncrease = 500
  if player == 1:
    x = 0
    y = 1
  elif player == 2:
    x = 1
    y = 0

  targets = []
  if len(Field[x][y]) == 0:
    print("There are no monsters to target on the field.")
    return False
 
  target = max(Field[x][y], key=lambda monster: monster.total_atk)
  target.increaseBonusAtk(AttackIncrease)
  target.equipped.append('Reinforcements')
  print(target.name + " gains 500 ATK until the end of this turn.")
  return True

In [10]:
#Raj's Cards

def changeOfHeart(player, Field):
    #Targets 1 monster your opponent controls; take control of it until the End Phase.
    monsterOp = Field[1][0]
    monsterPl = Field[0][1]

    if len(monsterOp) == 0:
        print("No monsters on the opponents field")
        return False
    else:
        #takes the optimal monster from the opponent to add it to the players monster
        optimalMonster = max(monsterOp, key=lambda monster: monster.total_atk)
        optimalMonster.switched = True
        monsterPl.append(optimalMonster)
        monsterOp.remove(optimalMonster)
        

    return True

        

def waboku(player, Field):
    #You take no battle damage this turn. Your monsters cannot be destroyed by battle this turn.
    monsterOp = Field[1][0]

    if monsterOp == 0:
        print("No monsters on the opponents field")
        return False
    
    #put all opponent monsters in defense position so they cannot attack the player as they dont take any battle damage this turn
    for monster in monsterOp:
        monster.can_atk = False
    return True



def dragonCaptureJar(player, Field):
    #Change all face-up Dragon-Type monsters on the field to Defense Position,
    #also they cannot change their battle positions.
    if player == 1:
      x = 0
      y = 1
    else:
      x = 1
      y = 0

    monsterPl = Field[x][y]
    monsterOp = Field[y][x]

    dragonMonster = []
    opDragonMonster = []
    # Return None if nothin on Field
    if  len(monsterOp) == 0:
        return False
    
    if len(monsterOp) > 0:
      
      for monster in Field[y][x]:
        if monster.race.lower() == "dragon":
          opDragonMonster.append(monster)

      if len(dragonMonster) == 0:
        return False

      else:
        if len(monsterPl) > 0:
          for monster in Field[x][y]:
            if monster.race.lower() == "dragon":
              dragonMonster.append(monster)
        if len(dragonMonster) > 0:
          opOptimalMonster = max(opDragonMonster, key=lambda monster: monster.total_atk)
          optimalMonster = max(dragonMonster, key=lambda monster: monster.total_atk)
          if opOptimalMonster > optimalMonster:
            for monster in Field[x][y]:
              if monster.race.lower() == "dragon":
                  monster.can_atk = False
          else:
            return False

        for monster in Field[y][x]:
          if monster.race.lower() == "dragon":
            monster.can_atk = False
            
   

    #Change all face up dragon monsters to defense positions

    return True





In [108]:
p1LPDiff = 0
p2LPDiff = 0
p1Wins = 0
p2Wins = 0
for x in range(1, 1001):
  print("Test Run:", x)
  p1LP, p2LP = letsDuel(p1Deck, p2Deck)
  if p1LP > p2LP:
    p1LPDiff = p1LP - p2LP
    p1Wins += 1
  elif p1LP < p2LP:
    p2LPDiff = p2LP - p1LP
    p2Wins += 1
  print("Test Run:", x, "Successful")
  print()
  print()
  print()

p1Avg = p1LPDiff/p1Wins
p2Avg = p2LPDiff/p2Wins
print("Player 1 Won", p1Wins, "by an Average of", p1Avg, "Points")
print("Player 2 Won", p2Wins, "by an Average of", p2Avg, "Points")
if p1Wins > p2Wins:
  print()
  print("Player 1 Wins", p1Wins/(p1Wins + p2Wins), "Percent of the Time")
elif p1Wins < p2Wins:
  print()
  print("Player 2 Wins", p2Wins/(p1Wins + p2Wins), "Percent of the Time")

Streaming output truncated to the last 5000 lines.
Kojikocy (1500 atk, 1200 def) Attacks Opponent Directly
Attack Successful
Opponent loses 1500 Life Points

Main Phase 2...
Updating Strategy... (
)...Strategy Updated

End Phase...
End Turn

P1 LP = 2600
P2 LP = 2550


Player 1's Turn:

Draw Phase...
Draw Ultimate Offering

Standby Phase...
Strategizing. Please Wait...
Strategy (
Set Trap(s)
) Strategy Found: No Monsters. Rely On Spell/Traps

Main Phase...

Battle Phase...

Main Phase 2...
Updating Strategy... (
)...Strategy Updated
Set 1 Trap(s)

End Phase...
End Turn

P1 LP = 2600
P2 LP = 2550


Player 2's Turn:

Draw Phase...
Draw Invigoration

Standby Phase...
Strategizing. Please Wait...
Strategy (
Equipping 'Invigoration' to Kojikocy to increase its attack by 400 and decrease its defense by -200
) Strategy Found: Advantage is Mine, Play Offense

Main Phase...
Summon Dark Titan of Terror (1300 atk, 1100 def)
Activate Invigoration
Equipping 'Invigoration' to Kojikocy to increase it